In [4]:
import pandas as pd
import io
import matplotlib 
import plotly.express as px
import plotly.graph_objects as go


In [12]:

mouse_means0 = pd.read_csv('https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_mouse_ctx-hip_smart-seq/trimmed_means.csv')
#human_means0 = pd.read_csv('https://idk-etl-prod-download-bucket.s3.amazonaws.com/aibs_human_ctx_smart-seq/trimmed_means.csv')

# Get all gene and cell type names: HUMAN
#human_idx = list(human_means0.columns.values)
#human_listOfColumnNames = list(human_means0['cluster_label'])
# Get all gene and cell type: MOUSE
mouse_idx = list(mouse_means0.columns.values)
mouse_listOfColumnNames = list(mouse_means0['cluster_label'])


#combine clusters into higher_clusters
newlist =[]
for i in mouse_listOfColumnNames:
    i = str(i)
    head, sep, tail = i.partition('_') 
    newlist.append(tail) 
mouse_means0['higher_cluster'] = newlist



#list what cell clusters count as which cell types
excite_list = ['CA1','CA1-ProS','CA2-IG-FC','CA3','CT SUB','DG', 'L2 IT ENTl',
 'L2 IT ENTm','L2 IT HATA','L2 IT PAR','L2 IT ProS','L2 IT RSP-ACA','L2 IT RSPv',
 'L2/3 IT APr','L2/3 IT CTX','L2/3 IT ENTl','L2/3 IT PPP','L2/3 IT TPE',
 'L3 IT ENTl','L3 IT ENTm','L3 RSP-ACA','L4/5 IT CTX','L4/5 IT TPE-ENT',
 'L5 IT CTX','L5 IT TPE-ENT','L5 NP CTX','L5 PPP','L5 PT CTX','L5 PT RSP-ACA',
 'L5/6 IT CTX', 'L6 Car3', 'L6 CT CTX','L6 CT ENTm','L6 IT CTX','L6 IT ENTl',
 'L6 IT RHP','L6 NP CT CTX','L6b CTX','L6b RHP','L6b/CT ENT','Pax6','SUB','SUB-ProS', 'ProS','Ndnf HPF'
 'NP PPP','NP SUB']

non_neuron_list = [ 'VLMC', 'Oligo', 'Micro', 'Endo', 'Astro', 'CR', 'OPC', 'Peri',  'PVM', 'SMC', 'v3d']

inhib_list =  ['Lamp5 Lhx6','Lamp5','Meis2','Pvalb Vipr2','Pvalb','Sncg','Sst Chodl','Sst','Vip']

all_list = excite_list + non_neuron_list + inhib_list

#add cell type data to frame
mega_grps = []
for i in mouse_means0["higher_cluster"]:
    if i in excite_list :
        mega_grps.append("Excitatory")
    if i in inhib_list :
        mega_grps.append("Inhibitory")
    if i in non_neuron_list :
        mega_grps.append("Non-Neuronal")
    if i not in all_list :
        mega_grps.append("Unknown")
mouse_means0["Cell_Type"] = mega_grps

#reshape for graphing
genestestdf1 = pd.melt(mouse_means0, id_vars =['cluster_label','higher_cluster', "Cell_Type"],
                      var_name ='Gene', value_name ='Expression')
genestestdf2 = genestestdf1[genestestdf1["Gene"] == 'Rapgef2']


fig = px.bar(genestestdf2, x="higher_cluster", y="Expression", color = "Cell_Type", 
                    #facet_col = "Cell_Type",
             #facet_wrap = 4,
                    width=1000, height=600,
                    hover_name = "cluster_label",
                    hover_data = ["higher_cluster", "Cell_Type", "Expression"]
                    )
for a in fig.layout.annotations:
    a.text = a.text.split("=")[1]
fig.update_yaxes(title_text='')
fig.update_layout(
    title="Rapgef2: Adult mouse, Single Cell, Allen Transcriptomics",
    xaxis_title="Cell Type",
    yaxis_title="Expression",
    xaxis={'categoryorder':'total descending'}
)
fig.show()

            ## print table
fig2 = go.Figure(data=[go.Table(
    header=dict(values=list(genestestdf2.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[genestestdf2.cluster_label,genestestdf2.higher_cluster, genestestdf2.Cell_Type, genestestdf2.Gene,genestestdf2.Expression],
               fill_color='lavender',
               align='left'))
])

fig2.update_layout(title="Rapgef2: Rapgef2: Adult mouse, Single Cell, Allen Transcriptomics")
fig2.show()

In [3]:
data = pd.read_csv("clusters.csv")
data.columns = ['Region','Class','Cluster','Rapgef2_Amount','Rapgef2_PVal']

fig = px.bar(data, x="Class", y="Rapgef2_Amount", color = "Class", 
                    #facet_row = "Region",
                     width=1000, height=800,
                    hover_name = "Cluster",
                    hover_data = ["Class", "Cluster", "Rapgef2_Amount", "Rapgef2_PVal"]
                    )
for a in fig.layout.annotations:
    a.text = a.text.split("=")[1]
fig.update_yaxes(title_text='')
fig.update_layout(
    title="Rapgef2: Adult mouse, Single Cell, Saunder, 2018 (DropViz)",
    xaxis_title="Cell Type",
    yaxis_title="Expression",xaxis={'categoryorder':'total descending'})
fig.show()

            ## print table
fig2 = go.Figure(data=[go.Table(
header=dict(values=list(data.columns),
                    fill_color='paleturquoise',
                    align='left'),
cells=dict(values=[data.Region,data.Class, data.Cluster, data.Rapgef2_Amount,
                              data.Rapgef2_PVal],
                    fill_color='lavender',
                    align='left'))
                ])
fig2.update_layout(
    title="Rapgef2: Adult mouse, Single Cell, Saunder, 2018 (DropViz)")

fig2.show()

In [17]:
data = pd.read_csv("LooSimon_MouseCortexDropSeq_E14P0_avgExpByCellType.csv")
# Get column names 
listOfColumnNames = list(data.columns.values[1:])
idx = list(data['GeneID'])



genes = ['Rapgef2']
cells = listOfColumnNames
data_gene_cell = data[cells]
data_gene_cell.index= data['GeneID']
data_gene_cell = data_gene_cell.loc[genes]
data_gene_cell = data_gene_cell.reset_index()

data_gene_cell2 = pd.melt(data_gene_cell, id_vars =['GeneID'],
                              var_name ='Cluster', value_name ='Expression')

ages = []
cell_type = []
supergroup = []

import re
for i in data_gene_cell2['Cluster']:    
    if '-P]' in i:
        ages.append('p0')
    if '-E]' in i:
        ages.append('e14.5')
    if 'Layer' in i:
        supergroup.append("Cortical Layers")
    if any(x in i for x in ['Endothelial','Choroid','Astrocyte','Oligo','Ganglionic','Micro']): 
        supergroup.append("Non-Neuronal")
    if any(x in i for x in ['SVZ','RG']): 
        supergroup.append("Proliferative")
    if "Int" in i:
        supergroup.append("Inhibitory")
    if any(x in i for x in ['Thalamic','Striatal']): 
        supergroup.append("Midbrain")
    j = re.sub("[\[].*?[\]]", "", i)
    cell_type.append(j)

data_gene_cell2['Cell_Type'] = cell_type
data_gene_cell2['Age'] = ages
data_gene_cell2['Category'] = supergroup


fig = px.bar(data_gene_cell2, x="Category", y="Expression", color = "Cell_Type", 
             facet_col = "Age",
             hover_name = "Cell_Type",
             hover_data = ["Age", "Cluster"]
            )
#fig.update_layout(barmode='group')
for a in fig.layout.annotations:
    a.text = a.text.split("=")[1]
fig.update_layout(
    title="Rapgef2: E14.5 and P0, Single Cell, Simon and Loo",
    xaxis_title="Category",
    yaxis_title="Expression")
fig.show()

fig2 = go.Figure(data=[go.Table(
        header=dict(values=list(data_gene_cell2.columns),
                fill_color='paleturquoise',
                align='left'),
        cells=dict(values=[data_gene_cell2.GeneID, data_gene_cell2.Cluster, data_gene_cell2.Expression,
                          data_gene_cell2.Cell_Type, data_gene_cell2.Age, data_gene_cell2.Category],
                fill_color='lavender',
                align='left'))
            ])
fig2.update_layout(
    title="Rapgef2: E14.5 and P0, Single Cell, Simon and Loo")
fig2.show()

In [7]:

df1 = pd.read_excel(r'C:\Users\dbaka\OneDrive\Desktop\Eiden lab\Python Scripts\Rapgef2 Datasets\ENCODE_Rapgef2_geneLevel.xlsx')
df1.sort_values(by=['Age'], inplace=True)


fig = px.bar(df1, x="Tissue", y="value", color = "Age", 
                    facet_col = "variable",
             #facet_wrap = 4,
                    width=1000, height=600,
                    #hover_name = "cluster_label",
                    #hover_data = ["higher_cluster", "Cell_Type", "Expression"]
                    )
for a in fig.layout.annotations:
    a.text = a.text.split("=")[1]
fig.update_yaxes(title_text='')
fig.update_layout(
    title="Rapgef2: Poly-A Oligo-dt primed, B Wold, ENCODE",
    xaxis_title="Cell Type",
    yaxis_title="Expression",
    xaxis={'categoryorder':'total descending'})
fig.show()

            ## print table
fig2 = go.Figure(data=[go.Table(
    header=dict(values=list(df1.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df1.Age,df1.Tissue, df1.variable, df1.value],
               fill_color='lavender',
               align='left'))
])


fig2.update_layout(title="Rapgef2: Poly-A Oligo-dt primed, B Wold, ENCODE")
fig2.show()
